In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "../include/")
import mongo_helper_functions as mongo_helper

from zipfile import ZipFile
import requests
import io
import pandas as pd
import json

In [2]:
national_drug = mongo_helper.connectMongo(
    host= "angst.soic.indiana.edu",
    db_name= "smm4h",
    collection_name= "national_drug_product"
)
national_drug.estimated_document_count()

0

In [3]:
zip_url = "https://www.accessdata.fda.gov/cder/ndcxls.zip"

In [4]:
r = requests.get(zip_url)
z = ZipFile(io.BytesIO(r.content))
z.namelist()

['package.xls', 'product.xls']

In [5]:
package_df = pd.read_csv(z.open(z.namelist()[0]), sep="\t")
package_df.info()
package_df.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240211 entries, 0 to 240210
Data columns (total 8 columns):
PRODUCTID             240211 non-null object
PRODUCTNDC            240211 non-null object
NDCPACKAGECODE        240211 non-null object
PACKAGEDESCRIPTION    240211 non-null object
STARTMARKETINGDATE    240184 non-null float64
ENDMARKETINGDATE      5491 non-null float64
NDC_EXCLUDE_FLAG      240211 non-null object
SAMPLE_PACKAGE        240211 non-null object
dtypes: float64(2), object(6)
memory usage: 14.7+ MB


,PRODUCTID,PRODUCTNDC,NDCPACKAGECODE,PACKAGEDESCRIPTION,STARTMARKETINGDATE,ENDMARKETINGDATE,NDC_EXCLUDE_FLAG,SAMPLE_PACKAGE
0,0002-0800_4bb5d1cb-0fa7-48c7-9f6d-8d45f9b91649,0002-0800,0002-0800-01,1 VIAL in 1 CARTON (0002-0800-01) > 10 mL in ...,19870710.0,NaN,N,N
1,0002-1200_957ee1b5-dfa7-4e3f-96e1-6bed1ffc0abe,0002-1200,0002-1200-30,"1 VIAL, MULTI-DOSE in 1 CAN (0002-1200-30) > ...",20120601.0,NaN,N,N


In [6]:
product_df = pd.read_csv(z.open(z.namelist()[1]), sep="\t", encoding = "ISO-8859-1")
product_df.info()
print(product_df.PRODUCTID.nunique())
product_df.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125719 entries, 0 to 125718
Data columns (total 20 columns):
PRODUCTID                           125719 non-null object
PRODUCTNDC                          125719 non-null object
PRODUCTTYPENAME                     125719 non-null object
PROPRIETARYNAME                     125715 non-null object
PROPRIETARYNAMESUFFIX               17186 non-null object
NONPROPRIETARYNAME                  125712 non-null object
DOSAGEFORMNAME                      125719 non-null object
ROUTENAME                           123629 non-null object
STARTMARKETINGDATE                  125714 non-null float64
ENDMARKETINGDATE                    3936 non-null float64
MARKETINGCATEGORYNAME               125719 non-null object
APPLICATIONNUMBER                   107919 non-null object
LABELERNAME                         125719 non-null object
SUBSTANCENAME                       123145 non-null object
ACTIVE_NUMERATOR_STRENGTH           123144 non-null object
ACTIV

,PRODUCTID,PRODUCTNDC,PRODUCTTYPENAME,PROPRIETARYNAME,PROPRIETARYNAMESUFFIX,NONPROPRIETARYNAME,DOSAGEFORMNAME,ROUTENAME,STARTMARKETINGDATE,ENDMARKETINGDATE,MARKETINGCATEGORYNAME,APPLICATIONNUMBER,LABELERNAME,SUBSTANCENAME,ACTIVE_NUMERATOR_STRENGTH,ACTIVE_INGRED_UNIT,PHARM_CLASSES,DEASCHEDULE,NDC_EXCLUDE_FLAG,LISTING_RECORD_CERTIFIED_THROUGH
0,0002-0800_4bb5d1cb-0fa7-48c7-9f6d-8d45f9b91649,0002-0800,HUMAN OTC DRUG,Sterile Diluent,NaN,diluent,"INJECTION, SOLUTION",SUBCUTANEOUS,19870710.0,NaN,NDA,NDA018781,Eli Lilly and Company,WATER,1,mL/mL,NaN,NaN,N,20191231.0
1,0002-1200_957ee1b5-dfa7-4e3f-96e1-6bed1ffc0abe,0002-1200,HUMAN PRESCRIPTION DRUG,Amyvid,NaN,Florbetapir F 18,"INJECTION, SOLUTION",INTRAVENOUS,20120601.0,NaN,NDA,NDA202008,Eli Lilly and Company,FLORBETAPIR F-18,51,mCi/mL,"Radioactive Diagnostic Agent [EPC],Positron Em...",NaN,N,20191231.0


In [8]:
national_drug.insert_many(json.loads(product_df.to_json(orient='records').lower()))